<a href="https://colab.research.google.com/github/atanilson/Comp702/blob/main/COMP702_Aplied.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geospacial Data Processing Tools

* Google Earth Engine, Python Earth Engine Application
* GeoPanda Geospacial Analysis
* Rasterio
* Foliom

In [ ]:
%%capture
!pip -q install --upgrade folium
!apt install libspatialindex-dev
!pip -q install rtree
!pip -q install geopandas
!pip -q install geojson
!pip -q install geemap==0.17.3
!pip -q uninstall tornado -y
!yes | pip install tornado==5.1.0
!pip -q install rasterio
!pip -q install tqdm
!pip -q install eeconvert

In [ ]:
# Standard imports
import os
from tqdm.auto import tqdm
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospacial processing packages
import geopandas as gpd
import geojson

import shapely
import rasterio as rio
from rasterio.plot import show
import rasterio.mask
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import geemap.eefolium as emap
import folium

# Model
import torch
from torchvision import datasets, models, transforms